# Chapter 05 - Exercises

### Exercise 5.6

**Q**

What is the equation analogous to (5.6) for action values Q(s, a) instead of state values V (s), again given returns generated using b?

**A**

The equation 5.6 is:

\begin{align*}
V(s) \stackrel{.}{=} \frac{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1}}
\end{align*}

And:

\begin{align*}
\rho_{t:T(t)-1} \stackrel{.}{=} \frac{\prod_{k=t}^{T-1} \pi(A_k | S_k) p(S_{k+1} | S_k, A_k)}{\prod_{k=t}^{T-1} b(A_k | S_k) p(S_{k+1} | S_k, A_k)} = \frac{\prod_{k=t}^{T-1} \pi(A_k | S_k)}{\prod_{k=t}^{T-1} b(A_k | S_k)}
\end{align*}

For $Q(s, a)$, we have $A_t=a$, so:

\begin{align*}
Q(s, a) &\stackrel{.}{=} \frac{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:t} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:t} \rho_{t+1:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(A_t | S_t)}{b(A_t | S_t)} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(A_t | S_t)}{b(A_t | S_t)} \rho_{t+1:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(a | s)}{b(a | s)} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(a | s)}{b(a | s)} \rho_{t+1:T(t)-1}} \\
&= \frac{\frac{\pi(a | s)}{b(a | s)} \sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1} G_t}{\frac{\pi(a | s)}{b(a | s)} \sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1}}
\end{align*}

In which $\mathcal{T}(s, a)$ are the occurences (or first occurence in the episode for first-visit Monte Carlo) of the state s and action a across all episodes.

Because $A_t=a$, the first term in the product that generate $\rho_{t:T(t)-1}$ is a constant, and can be isolated outside the sum. 

In a weighted importance sampling (which is the case above), the constant is present both in the numerator and denominator and cancel each other, that is, the importance-sampling ratio, $\rho$, can consider only the times from $t+1$ onwards in a weighted importance sampling for $Q(s, a)$.



### Exercise 5.7

**Q**

In learning curves such as those shown in Figure 5.3 error generally decreases with training, as indeed happened for the ordinary importance-sampling method. But for the weighted importance-sampling method error first increased and then decreased. Why do you think this happened?

**A**

As defined in the example:

- The dealer is showing a deuce.
- The sum of the player's card is 13.
- The player has a usable ace.
- The target policy is to stick only on a sum of 20 or 21.
- The behavior policy is to choose to hit or stick randomly.
- The error for each amount of episodes is averaged among 100 independent runs.

An increased mean squared error means that the calculated value for the state was more distant from the target. In this state, we have an ace, which is expected to show different behaviors depending on whether you reach 20/21 and stop, or exceed, changing the value of the ace from 11 to 1 and trying to reach 20/21 again, which can be seen as a 2nd chance to reach the expected sum.

The values are estimated from the behavior policy, scaling the returns by the importance-sampling ratio ($\rho$), and then dividing by the sum of the ratios (weighted average). Because the target policy only accepts time-steps that ends in 20 or 21 (stopping when reaching any of these states), or exceeds (because the new card value makes the sum exceed 21), all other cases are dismissed (the ratio is 0). Also, the target policy is deterministic ($\pi(hit|s) = 1$ when $sum < 20$, $\pi(stick|s) = 1$ when $20 <= sum$) while the behavior policy for $n-1$ hits and 1 stick (ending the episode), or n hits, exceeding 21 and losing, has n time-steps and has a probability of happening equal to $2^{-n}$, because it's a random policy with 2 actions, so $b(a|s) = 0.5$ for any $a \in \mathcal{A}(s) = \{hit, stick\}$. 

This means that for a higher amount of hits, the probability of the random policy actually executing those actions reduces exponentially, maybe requiring the completion of a huge number of episodes to simulate one such case.

In the given state, the player has an ace, but the number of time-steps to end the episode when 20 or 21 is reached directly is smaller than the number of time-steps if it's exceeded and the ace has its value changed to 1, trying again and reaching 20 or 21, or exceeding 21 and losing. The cases in which the player loses will, on average, have more time-steps (due to more hits). It's possible that many (or even all) among the 100 independent runs of the episodes in the first attempts never reach the case in which the player loses, choosing to stick before (even in cases in which the player has sum less than 20, which will be ignored in the ratio), or exceeds 21 after the player reaches 20 or 21 (in which it would have stopped in the target policy, so not considered in the ratio), making the state have a value higher than it should have.

It eventually converges in the expected value as more episodes are completed, and more runs have cases in which the player loses that can happen in the target policy. 

It's important to note that cases in which the player exceeds 21 will have a much lower probability of happening than cases in which the player sticks to 20/21 using the behavior (random) policy, and because the probability for the target policy will be 1 (in the valid cases in which 20 or 21 was not reached in some previous step), the value of $\rho$ will be higher, giving more weight to these cases, evetually compensating the lower frequency.

*Note:* It's also possible that the behavior policy never reaches 20/21 in the first episodes, which would also influence the target policy value (it would be 0, because no sequence of actions across all runs using the behavior policy are possible under the target policy). Actually, this can happen for any initial states, even those without a usable ace. That said, considering that it's done an average across 100 independent runs, and reaching 20/21 can be done in very few steps, it's more likely to be the first case provided in this answer (there are no case in which the player exceeds 21, but there are cases in which the player stops at 20/21).

### Exercise 5.8

**Q**

The results with Example 5.5 and shown in Figure 5.4 used a first-visit MC method. Suppose that instead an every-visit MC method was used on the same problem. Would the variance of the estimator still be infinite? Why or why not?

**A**

Yes. The increase in value is exponential, while the every-visit MC would simply average the results of an episode accross the occurences, so the total would still diverges into infinite.

Mathematically, while the case for first visits can be defined as:

\begin{align*}
\mathbb{E}[X^2] &= \mathbb{E}_b[(\rho_{t:T(t)-1} G_0)^2] \\
&= \mathbb{E}_b[\rho_{t:T(t)-1}^2] \\
&= \mathbb{E}_b\left[\left(\prod_{t=0}^{T-1} \frac{\pi(A_t | S_t)}{b(A_t | S_t)}\right)^2\right] \\
&= \sum_{k=0}^{\infty} \left(b(left | s) \cdot p(s_T | s, left) \cdot \left(\frac{\pi(left | s)}{b(left | s)}\right)^2\right) \left(b(left | s)^k \cdot p(s | s, left)^k \cdot \left(\prod_{t=0}^k \frac{\pi(left | s)}{b(left | s)}\right)^2\right) \\
&= \sum_{k=0}^{\infty} \left(0.5 \cdot 0.1 \cdot \left(\frac{1}{0.5}\right)^2\right) \cdot \left(0.5^k \cdot 0.9^k \cdot \left(\prod_{t=0}^k \frac{1}{0.5}\right)^2\right) \\
&= \sum_{k=0}^{\infty} (0.5 \cdot 0.1 \cdot 2^2) \cdot \left(0.5^k \cdot 0.9^k \cdot \left(\prod_{t=0}^k 2^k\right)^2\right) \\
&= \sum_{k=0}^{\infty} 0.2 \cdot \left(\left(\frac{1}{2}\right)^k \cdot 0.9^k \cdot 2^{2k}\right) \\
&= 0.2 \sum_{k=0}^{\infty} 0.9^k \cdot \frac{2^{2k}}{2^k} \\
&= 0.2 \sum_{k=0}^{\infty} 0.9^k \cdot 2^k \\
&= 0.2 \sum_{k=0}^{\infty} 1.8^k \\
&= \infty
\end{align*}

The above is calculated considering only the cases in which the left action is chosen (the ratio, $\rho$, ignores the right action, because the target policy only chooses the left action). So it considers all possibilities, weighed by the probabilities of each occurence (the probability of choosing the action using the behaviour (random) policy, 0.5, and the probability of staying in the same state, 0.9, or ending the episode, 0.1), multiplied by what is expected, $\rho^2$. 

The possibilities are: 

- The first left action ends the episode with probability 0.1, k=0; 
- The first left action stays in the same state, probability 0.9, then the second left action ends the episode with probability 0.1, k=1; 
- And so on for k going to $\infty$, because it can stay in the same state indefinitely when choosing the left action.

For an every-visit MC, for a case in which the left action returned back n times, and in the n+1 time-step it ended the episode, instead of considering only the probability of the first occurence, $0.1 \cdot 0.9^n$, it will consider all cases and calculate the average, $\frac{\sum_{i=0}^n 0.1 \cdot 0.9^i}{n+1}$ (it's n+1 time-steps: n time-steps returning to the same position, and one last time-step ending the episode), so the variance is:

\begin{align*}
\mathbb{E}[X^2] &= \mathbb{E}_b[\rho_{t:T(t)-1}^2] \\
&= \mathbb{E}_b\left[\left(\prod_{t=0}^{T-1} \frac{\pi(A_t | S_t)}{b(A_t | S_t)}\right)^2\right] \\
&= 0.2 \sum_{k=0}^{\infty} \frac{\sum_{i=0}^k 1.8^i}{k} \\
&= 0.2 \sum_{k=0}^{\infty} \frac{1.8^k}{k} + 0.2 \sum_{k=0}^{\infty} \frac{\sum_{i=0}^{k-1} 1.8^i}{k} \\
&\geq 0.2 \sum_{k=0}^{\infty} \frac{1.8^k}{k} \\
&= \infty
\end{align*}

The above equation is infinite because the first term of the sum is infinite the second is positive, and it ends up in a sum of positive terms in which the highest term goes to infinity:

\begin{align*}
\operatorname*{lim}_{x \to \infty}{0.2 \sum_{k=0}^x \frac{1.8^k}{k}} \geq 0.2 \operatorname*{lim}_{x \to \infty}{\frac{1.8^x}{x}} = \infty
\end{align*}

In the above equation, the numerator grows exponentially, while the denominator grows linearly, both toward $\infty$, so the division is $\infty$. We can apply L'Hôpital's rule since both terms grow to infinity:

\begin{align*}
\operatorname*{lim}_{x \to \infty}{\frac{1.8^x}{x}} &= \operatorname*{lim}_{x \to \infty}{\frac{\frac{d(1.8^x)}{dx}}{\frac{dx}{dx}}} \\
&= \operatorname*{lim}_{x \to \infty}{\frac{1.8^x ln(1.8)}{1}} \\
&= ln(1.8) \operatorname*{lim}_{x \to \infty}{1.8^x} \\
& = \infty
\end{align*}

So, the variance of the estimator would still be infinite (the first of all the cases in an episode grows to infinity exponentially, averaged by the number of cases that also grow to infinity, but linearly, continues to diverge into infinity).

### Exercise 5.9

**Q**

Modify the algorithm for first-visit MC policy evaluation (Section 5.1) to use the incremental implementation for sample averages described in Section 2.4.

**A**

The original algorithm for first-visit MC policy evaluation is as follows:

> Input: a policy $\pi$ to be evaluated
>
> Initialize:<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$V(s) \in \mathbb{R}$, arbitrarily, for all $s \in \mathcal{S}$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$Returns(s) \gets$ an empty list, for all $s \in \mathcal{S}$
>
>Loop forever (for each episode):<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;Generate an episode following $\pi$: $S_0$, $A_0$, $R_1$, $S_1$, $A_1$, $R_2$, ..., $S_{T-1}$, $A_{T-1}$, $R_T$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$G \gets 0$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;Loop for each episode, t = T - 1, T - 2, ..., 0:<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$G \gets \gamma G + R_{t+1}$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Unless $S_t$ appears in $S_0$, $S_1$, ..., $S_{t-1}$:<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Append G to $Returns(S_t)$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$V(S_t) \gets average(Returns(S_t))$

The incremental implementation follows the form:

\begin{align*}
NewEstimate \gets OldEstimate + StepSize [Target  - OldEstimate] \tag{2.4}
\end{align*}

For example, the estimated action value $Q_n$ considering the average reward among the rewards received previously $R_1$, $R_2$, ..., $R_{n-1}$ is:

\begin{align*}
Q_n \stackrel{.}{=} \frac{R_1 + R_2 + ... + R_{n-1}}{n-1}
\end{align*}

The next action value follow the same approach, and can also be defined based only on the current reward and the previous value (instead of keeping track of all rewards):

\begin{align*}
Q_{n+1} = \frac{1}{n} \sum_{i=1}^n R_i = Q_n + \frac{1}{n}[R_n - Q_n] \tag{2.3}
\end{align*}

The modified algorithm is:

> Input: a policy $\pi$ to be evaluated
>
> Initialize:<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$V(s) \in \mathbb{R}$, arbitrarily, for all $s \in \mathcal{S}$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$Returns(s) \gets$ 0, for all $s \in \mathcal{S}$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$Amount(s) \gets$ 0, for all $s \in \mathcal{S}$
>
>Loop forever (for each episode):<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;Generate an episode following $\pi$: $S_0$, $A_0$, $R_1$, $S_1$, $A_1$, $R_2$, ..., $S_{T-1}$, $A_{T-1}$, $R_T$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;$G \gets 0$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;Loop for each episode, t = T - 1, T - 2, ..., 0:<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$G \gets \gamma G + R_{t+1}$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Unless $S_t$ appears in $S_0$, $S_1$, ..., $S_{t-1}$:<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$Amount(S_t) \gets Amount(S_t) + 1$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$Returns(S_t) \gets Returns(S_t) + \frac{1}{Amount(S_t)}[G - Returns(S_t)]$<br/>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$V(S_t) \gets Returns(S_t)$


### Exercise 5.10

**Q**

Derive the weighted-average update rule (5.8) from (5.7). Follow the pattern of the derivation of the unweighted rule (2.3).

**A**

The equation 5.7 is:

\begin{align*}
V_n \stackrel{.}{=} \frac{\sum_{k=1}^{n-1} W_k G_k}{\sum_{k=1}^{n-1} W_k}, n \geq 2
\end{align*}

The equation 5.8 is:

\begin{align*}
V_{n+1} \stackrel{.}{=} V_n + \frac{W_n}{C_n}[G_n - V_n], n \geq 1
\end{align*}

Following the pattern of the derivation of the unweighted rule (2.3), applied to the equation 5.7, we have:

\begin{align*}
V_{n+1} &\stackrel{.}{=} \frac{\sum_{k=1}^n W_k G_k}{\sum_{k=1}^n W_k}, n \geq 1 \tag{5.7, for n+1} \\
&= \frac{W_n G_n + \sum_{k=1}^{n-1} W_k G_k}{C_n} \\
&= \frac{W_n G_n + \left(\frac{\sum_{k=1}^{n-1} W_k}{\sum_{k=1}^{n-1} W_k}\right) \sum_{k=1}^{n-1} W_k G_k}{C_n} \\
&= \frac{W_n G_n + \left(\sum_{k=1}^{n-1} W_k\right) \frac{\sum_{k=1}^{n-1} W_k G_k}{\sum_{k=1}^{n-1} W_k}}{C_n} \\
&= \frac{W_n G_n + \left(\sum_{k=1}^{n-1} W_k\right) V_n}{C_n} \\
&= \frac{W_n G_n + \left(\left(\sum_{k=1}^{n-1} W_k\right) + W_n - W_n\right) V_n}{C_n} \\
&= \frac{W_n G_n + \left(\left(\sum_{k=1}^n W_k\right) - W_n\right) V_n}{C_n} \\
&= \frac{W_n G_n + (C_n - W_n) V_n}{C_n} \\
&= \frac{W_n G_n + C_n V_n - W_n V_n}{C_n} \\
&= V_n + \frac{W_n}{C_n} [G_n - V_n], n \geq 1 \tag{5.8}
\end{align*}


### Exercise 5.11

**Q**

In the boxed algorithm for o↵-policy MC control, you may have been expecting the W update to have involved the importance-sampling ratio $\frac{\pi(A_t|S_t)}{b(A_t|S_t)}$, but instead it involves $\frac{1}{b(A_t|S_t)}$. Why is this nevertheless correct?

**A**

The policy $\pi$ is deterministic, so there's exactly one action that has probability 1, returned by $\pi(S_t)$, in which case $\frac{\pi(A_t|S_t)}{b(A_t|S_t)} = \frac{1}{b(A_t|S_t)}$, and all other actions have probability 0, making the next W, and all next Ws (toward earlier episodes) 0, in which case it's better to simply end the loop and go to the next episode, because no new changes will be made to $C(S_t, A_t)$ and $Q(S_t, A_t)$ and, consequently, $\pi$.

### Exercise 5.12: Racetrack (programming)

**Q**

Consider driving a race car around a turn like those shown in Figure 5.5. You want to go as fast as possible, but not so fast as
to run o↵ the track. In our simplified racetrack, the car is at one of a discrete set of grid positions, the cells in the diagram. The velocity is also discrete, a number of grid cells moved horizontally and vertically per time step. The actions are increments to the velocity components. Each may be changed by +1, -1, or 0 in each step, for a total of nine (3 x 3) actions. Both velocity components are restricted to be nonnegative and less than 5, and they cannot both be zero except at the starting line. Each episode begins in one of the randomly selected start states with both velocity components zero and ends when the car crosses the finish line. The rewards are -1 for each step until the car crosses the finish line. If the car hits the track boundary, it is moved back to a random position on the starting line, both velocity components are reduced to zero, and the episode continues. Before updating the car’s location at each time step, check to see if the projected path of the car intersects the track boundary. If it intersects the finish line, the episode ends; if it intersects anywhere else, the car is considered to have hit the track boundary and is sent back to the starting line. To make the task more challenging, with probability 0.1 at each time step the velocity increments are both zero, independently of the intended increments. Apply a Monte Carlo control method to this task to compute the optimal policy from each starting state. Exhibit several trajectories following the optimal policy (but turn the noise off for these trajectories).


**A**

The state can be considered as the 4-value tupe (x, y, h, v) that has the values of the x position (from 0 to x_max), y position (from 0 to y_max), horizontal velocity (going right, to increase x, from 0 to 4) and vertical velocity (going up, to increase y, from 0 to 4).

The starting line is defined by y = 0, and the car starts stopped, so the initial state is (x, 0, 0, 0), with x being random but with a possible value (the racetrack may not be a rectangle, so there may be some x positions that has a y positions, while others are off bounds).

The finish line is y = y_max, and the episode ends when the car passes through it (even if the car were to end outside the track).

The action can be considered as the 2 value tuple (v_x, v_y) to increment horizontaly and verticaly the velocity of the car, with v_x and v_y possible values being -1, 0 and 1, but they cannot change the final velocity to 0 (h=0, v=0), so (v_x, v_y) must be different from (-h, -v).

To make the grid allow both cases shown in the Figure 5.5, it can be defined as a rectangle with 30 rows ($0 \leq y \leq 29$) and 32 columns ($0 \leq x \leq 31$).

It receive 2 lists with 30 elements each with integer values between 0 and 15 to define the cells to consider offtrack, one of the lists removing from the left the first n elements in the row r, with n being the value in the index r of the the list, and the other list removing from the right. The second list must have the last k elements with values 0, with $1 \leq k \leq 29$, which will define the finish line, and all other values must be greater than 0. It may also be able to receive custom lists (for example, to specify the racetrack equal to the figure).

For a given racetrack (possible values of x and y), the policy can be calculated. When using another racetrack, another policy must be used (because the states will be different).

In [122]:
import matplotlib.pyplot as plt
import numpy as np
import random

NORMAL_COLOR = np.array([1., 1., 1.])
START_COLOR = np.array([255, 99, 71]) / 255
FINISH_COLOR = np.array([152, 249, 152]) / 255
OUT_OF_BOUNDS_COLOR = np.array([0.5, 0.5, 0.5])
CAR_COLOR = np.array([39, 116, 218]) / 255

TYPE_NORMAL = 0
TYPE_START = 1
TYPE_FINISH = 2
TYPE_OUT_OF_BOUNDS = 3

class RacetrackState:
    def __init__(self, row: int, col: int, horizontal_v: int, vertical_v: int):
        self.row = row
        self.col = col
        self.horizontal_v = horizontal_v
        self.vertical_v = vertical_v

class RacetrackEnv:
    def __init__(self, size: tuple[int, int], out_of_bounds: tuple[list[int], list[int]], max_steps: int | None = None):
        rows, cols = size
        out_left, out_right = out_of_bounds

        assert rows > 2 and cols > 2, "Invalid grid size"
        assert len(out_left) == len(out_right) == rows, "Invalid out-of-bounds size"
        assert len([o == 0 for o in out_left]) >= 2, "Invalid start line (must have at least two cells at the bottom)"
        assert len([o == 0 for o in out_right]) >= 2, "Invalid finish line (must have at least two cells at the right)"
        assert out_left[rows-1] > 0, "Invalid start line (it must not react the rightmost column)"
        assert out_right[0] > 0, "Invalid finish line (it must not react the bottom row)"

        self.contiguous_zeros(out_left, "Invalid start line (must be contiguous)")
        self.contiguous_zeros(out_right, "Invalid finish line (must be contiguous)")

        assert all(o >= 0 and o <= (cols - 1) for o in out_left), "Invalid left out-of-bounds definition (must be in the left half)"
        assert all(o >= 0 and o <= (cols - 1) for o in out_right), "Invalid right out-of-bounds definition (must be in the right half)"

        self.rows = rows
        self.cols = cols
        self.grid = np.full((self.rows, self.cols), TYPE_NORMAL)

        for row, o in enumerate(out_left):
            self.grid[row, :o] = TYPE_OUT_OF_BOUNDS

        for row, o in enumerate(out_right):
            self.grid[row, cols-o:] = TYPE_OUT_OF_BOUNDS

        for col in range(cols):
            if self.grid[0, col] != TYPE_OUT_OF_BOUNDS:
                self.grid[0, col] = TYPE_START

        for row in range(rows):
            if self.grid[row, cols-1] != TYPE_OUT_OF_BOUNDS:
                self.grid[row, cols-1] = TYPE_FINISH

        self.state: RacetrackState | None = None
        self.max_steps = max_steps

    def start(self, seed: int = 0) -> tuple[int, int, int, int]:
        row = 0
        valid_cols = [i for i, type in enumerate(self.grid[row]) if type == TYPE_START]
        random.seed(seed)
        col: int = random.choice(valid_cols)
        horizontal_v = 0
        vertical_v = 0

        state = RacetrackState(row, col, horizontal_v, vertical_v)
        self.state = state

        return state

    def reset(self, seed: int = 0) -> tuple[int, int, int, int]:
        state = self.start(seed)
        self.steps = 0
        return state

    def step(self, action: tuple[int, int]) -> tuple[RacetrackState, int, bool, bool]:
        row, col, horizontal_v, vertical_v = self.state
        horizontal_v_change, vertical_v_change = action

        assert horizontal_v_change >= -1 and horizontal_v_change <= 1, "Invalid row move"
        assert vertical_v_change >= -1 and vertical_v_change <= 1, "Invalid col move"

        new_horizontal_v = horizontal_v + horizontal_v_change
        new_vertical_v = vertical_v + vertical_v_change

        assert new_horizontal_v != 0 and new_vertical_v != 0, "Invalid acceleration (can't stop)"
        assert new_horizontal_v >= -5 and new_horizontal_v <= 5, "Invalid acceleration (horizontal)"
        assert new_vertical_v >= -5 and new_vertical_v <= 5, "Invalid acceleration (vertical)"

        new_row = row + new_horizontal_v
        new_col = col + new_vertical_v

        if new_row < 0 or new_row >= self.rows or new_col < 0 or new_col >= self.cols:
            reward = -1
            new_row, new_col, new_horizontal_v, new_vertical_v = self.start()
        elif self.grid[new_row, new_col] == TYPE_OUT_OF_BOUNDS:
            reward = -1
            new_row, new_col, new_horizontal_v, new_vertical_v = self.reset()
        else:
            rows_path = range(min(row, new_row), max(row, new_row) + 1)
            cols_path = range(min(col, new_col), max(col, new_col) + 1)
            projection_path = [(r, c) for r in rows_path for c in cols_path]
            reached_finish = any(self.grid[r, c] == TYPE_FINISH for r, c in projection_path)

            if reached_finish:
                reward = 1
                new_row, new_col, new_horizontal_v, new_vertical_v = self.reset()
            else:
                reward = 0

        state = RacetrackState(new_row, new_col, new_horizontal_v, new_vertical_v)
        self.state = state

        truncated = self.max_steps is not None and self.max_steps <= 0

        return state, reward, reward == 1, truncated

    @staticmethod
    def contiguous_zeros(arr: list[int], error_msg: str) -> bool:
        rows = len(arr)
        reached_zero = False
        reached_after_zero = False

        for i in range(rows):
            if not reached_zero:
                if arr[i] == 0:
                    reached_zero = True
            elif not reached_after_zero:
                if arr[i] != 0:
                    reached_after_zero = True
            else:
                assert arr[i] != 0, error_msg

    def plot(self, moves: list[tuple[int, int]]):
        rows, cols = self.rows, self.cols

        color_grid = np.full((rows, cols, 3), NORMAL_COLOR)

        for row in range(rows):
            inv_row = rows - row - 1
            for col in range(cols):
                if self.grid[row, col] == TYPE_START:
                    color_grid[inv_row, col] = START_COLOR
                elif self.grid[row, col] == TYPE_FINISH:
                    color_grid[inv_row, col] = FINISH_COLOR
                elif self.grid[row, col] == TYPE_OUT_OF_BOUNDS:
                    color_grid[inv_row, col] = OUT_OF_BOUNDS_COLOR

        plt.figure(figsize=(12, 12))
        plt.imshow(color_grid, aspect='auto')
        plt.grid(which='both', color='#333', linestyle='-', linewidth=1)
        plt.xticks(np.arange(.5, self.cols, 1), [])
        plt.yticks(np.arange(.5, self.rows, 1), [])
        plt.tick_params(bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)

        for move in moves:
            row_move, col_move = move
            row_move = rows - row_move - 1
            plt.plot(col_move, row_move, 'o', color=CAR_COLOR, markersize=10)

        plt.show()

    @classmethod
    def case_01(cls):
        rows, cols = 32, 18
        left_offset = 1
        out_left = [3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 3]
        out_left = [left_offset + o for o in out_left]
        out_right = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 0, 0, 0, 0, 0]
        return cls((rows, cols), (out_left, out_right))

    @classmethod
    def case_02(cls):
        rows, cols = 30, 32
        left_offset = 1
        out_left = [0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 14, 14, 14, 14, 13, 12, 11, 11, 11, 11, 12, 13, 16]
        out_left = [left_offset + o for o in out_left]
        out_right = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 6, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        return cls((rows, cols), (out_left, out_right))

# env = RacetrackEnv.case_01()
# env.plot(moves=[(1, 1), (3, 2)])

# env = RacetrackEnv.case_02()
# env.plot(moves=[(1, 1), (3, 2)])